In [1]:
import argparse
from enum import Enum
import sys
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import NoSuchElementException

import time
import random

In [2]:
driver = webdriver.Firefox()

The geckodriver version (0.35.0) detected in PATH at C:\Users\alejandro.regueiroru\.wdm\drivers\geckodriver\win64\v0.35.0\geckodriver.exe might not be compatible with the detected firefox version (138.0.3); currently, geckodriver 0.36.0 is recommended for firefox 138.*, so it is advised to delete the driver in PATH and retry


In [3]:
def waitFinishLoad(driver):
    time.sleep(0.1)

In [4]:
def clickByXPath(auxText, driver):
    waitFinishLoad(driver)
    elem = driver.find_element(By.XPATH, auxText)
    elem.click()
    waitFinishLoad(driver)

In [5]:
Minimun_wage_path = "https://rshiny.ilo.org/dataexplorer55/?lang=en&segment=indicator&id=EAR_4MMN_CUR_NB_A"

In [6]:
driver.get(Minimun_wage_path)

In [7]:
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//button[@data-id='filter_classif1']")))

<selenium.webdriver.remote.webelement.WebElement (session="0055dc04-9a99-4c5f-ae99-885acc23b2aa", element="498067d2-5407-483a-9554-1635efecd7e1")>

In [8]:
driver.find_element(By.XPATH, "//button[@data-id='filter_classif1']").click()
driver.find_element(By.ID, "bs-select-5-0").click()
driver.find_element(By.ID, "bs-select-5-1").click()
driver.find_element(By.XPATH, "//button[@data-id='filter_classif1']").click()

In [9]:
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//td[normalize-space(text())='Afghanistan']")))

<selenium.webdriver.remote.webelement.WebElement (session="0055dc04-9a99-4c5f-ae99-885acc23b2aa", element="c11d708e-2c9f-44e2-a2c0-e9b88253c879")>

In [10]:

# Localiza los controladores (handles) de la barra
controlador_izquierdo = driver.find_element(By.CSS_SELECTOR, ".irs-handle.from")
controlador_derecho = driver.find_element(By.CSS_SELECTOR, ".irs-handle.to")

# Calcula el desplazamiento necesario para llevar el controlador izquierdo al inicio
offset_izquierda = -150

# Calcula el desplazamiento necesario para llevar el controlador derecho al final
offset_derecha = 0

acciones = ActionChains(driver)

# Desplaza el controlador izquierdo completamente a la izquierda
acciones.drag_and_drop_by_offset(controlador_izquierdo, offset_izquierda, 0).perform()
print("Controlador izquierdo desplazado a la izquierda.")

# Espera un momento para que el desplazamiento se complete
time.sleep(1)

# Desplaza el controlador derecho completamente a la derecha
acciones.drag_and_drop_by_offset(controlador_derecho, offset_derecha, 0).perform()
print("Controlador derecho desplazado a la derecha.")

Controlador izquierdo desplazado a la izquierda.
Controlador derecho desplazado a la derecha.


In [11]:
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//option[@value='500']")))

<selenium.webdriver.remote.webelement.WebElement (session="0055dc04-9a99-4c5f-ae99-885acc23b2aa", element="b4680fc9-2c16-411e-8d62-6459fd645346")>

In [12]:
driver.find_element(By.XPATH, "//option[@value='500']").click()

In [13]:
tabla = driver.find_element(By.CSS_SELECTOR, ".display.compact.dataTable.no-footer")

In [14]:
import pandas as pd
data_list = []
while True:
    # Extraemos el body de la tabla
    tabla_body = tabla.find_element(By.TAG_NAME, "tbody")

    # Extraemos todas las filas de la tabla y las guardamos en una lista
    filas = tabla_body.find_elements(By.TAG_NAME, "tr")
    
    for fila in filas:
        columns = fila.find_elements(By.TAG_NAME, "td")
        
        # Crear un objeto (diccionario en este caso) con los datos de la fila
        if len(columns) == 5:  # Asegurarse de que la fila tiene las 5 columnas esperadas
            row_data = {
                'Reference area': columns[0].text,
                'Source': columns[1].text,
                'Currency': columns[2].text,
                'Year': columns[3].text,
                'Value': columns[4].text
            }
            data_list.append(row_data)

    # Si el número de filas es menor a 500, terminamos el bucle
    if len(filas) < 500:
        break

    driver.find_element(By.CSS_SELECTOR, ".paginate_button.next").click()

print("Fin nominas")

# Crear un DataFrame de pandas con los datos extraídos
df = pd.DataFrame(data_list)

Fin nominas


In [15]:
df

,Reference area,Source,Currency,Year,Value
0,Afghanistan,ADM - Labour legislation,U.S. dollars,2020,71.6
1,Afghanistan,ADM - Labour legislation,U.S. dollars,2019,70.8
2,Afghanistan,ADM - Labour legislation,U.S. dollars,2018,76.3
3,Afghanistan,ADM - Labour legislation,U.S. dollars,2017,80.8
4,Afghanistan,ADM - Labour legislation,U.S. dollars,2016,73.7
...,...,...,...,...,...
3552,Zambia,ADM - Labour legislation,U.S. dollars,2011,86.4
3553,Zambia,ADM - Labour legislation,U.S. dollars,2010,55.9
3554,Zambia,ADM - Labour legislation,U.S. dollars,2009,53.1
3555,Zambia,ADM - Labour legislation,U.S. dollars,2008,71.6


In [16]:
df.to_csv('../csv/minimun-wage.csv', index=False)